In [1]:
import PacTimeOrig.DataHandling as DH
import PacTimeOrig.Methods.utils as pacutils
from PacTimeOrig.Methods.Clusteresque import recusiveCDF, EMDRecursive,getvarforclustering,loadEMDmatrices,EMDfast

import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
from sklearn.cluster import AffinityPropagation

#set some analysis variables that'll stay fixed
winlength=10
n_sessions=5

In [2]:
# Load each pikcled datum and concatante the P1 trials
datall=dict()
for i in range(5):
    session=i+1
    file = open('C:\\Users\\Justin\\Documents\\GitHub\\PacManHuman\\Results\\datum'+str(session)+'.pkl', 'rb')
    emdmatrices = pickle.load(file)
    datall[str(session)]=emdmatrices

# Then compute recursive Earth movers across all and store matrices for clustering.
#Fewer bins
histrange=dict()

histrange['speed']=np.arange(0,1400+300,400) # 4bins
histrange['speedgrad']=np.arange(-200,200+50,100) # 4bins
histrange['accelmag']=np.arange(0,20000+5000,5000) # 4bins
histrange['prey1reldistance']=np.arange(0,2000+400,400) # 5bins
histrange['prey1reldistancegrad']=np.arange(-40,40+10,20) #4 bins
histrange['prey1relangularhead']= np.arange(-180,180+45,45)
histrange['prey1relangularheadgrad']= np.arange(-30,30+30,15)
histrange['prey1_scalar_projection']=np.arange(-1500,1500+500,500)
histrange['prey1_scalar_projection_gradient']=np.arange(-200,200+100,100)

emdmats=dict()


allvars=['speed','speedgrad','accelmag','prey1reldistance','prey1reldistancegrad','prey1relangularhead','prey1relangularheadgrad','prey1_scalar_projection','prey1_scalar_projection_gradient']


#Compute recursive EMD for all above variables. It's fast afuck now
for varname in allvars: #Loop over variables
    dattmp=[] #list of dataframes
    for i in range(n_sessions): #loop over sessions and retrieve data
        session=i+1
        dat=getvarforclustering(datall[str(session)]['p1trialdat'],varname,winlength)
        dattmp.append([pd.DataFrame(dat)])

    #Now combine all sessions
    datcombo=pd.DataFrame()

    for i in range(n_sessions):
        datcombo=pd.concat((datcombo,dattmp[i][0]),axis=1)

    #Now compute CDFs as prep for wasserstein distance
    recdf = recusiveCDF(datcombo.to_numpy(),histrange,varname=varname)
    #Now compute earthmovers distance
    emdmat=EMDfast(recdf,normalize=False)
    emdmatrices[varname]=emdmat


100%|██████████| 4/4 [00:12<00:00,  3.11s/it]


In [7]:
#Do affinity clustering
A=(emdmatrices['speed']*-1)/np.max(emdmatrices['speed'])
B=(emdmatrices['speedgrad']*-1)/np.max(emdmatrices['speedgrad'])
C=(emdmatrices['prey1_scalar_projection']*-1)/np.max(emdmatrices['prey1_scalar_projection'])
D=(emdmatrices['prey1_scalar_projection_gradient']*-1)/np.max(emdmatrices['prey1_scalar_projection_gradient'])

E=(emdmatrices['prey1relangularhead']*-1)/np.max(emdmatrices['prey1relangularhead'])

F=(emdmatrices['prey1reldistancegrad']*-1)/np.max(emdmatrices['prey1reldistancegrad'])

afmat=((A+B+C+D+F)/5)
af = AffinityPropagation(random_state=0,affinity='precomputed',damping=0.95).fit(afmat*-1)





import umap
import hdbscan
#UMAP plot with cluster labels
reducer = umap.UMAP(metric='precomputed',n_components=2,n_neighbors=25)
embedding=reducer.fit_transform(afmat*-1)

#OLD


afmat=((A+B+C+D+E+F)/6)
#afmat=((A+B+C+D+E)/5)



plt.scatter(embedding[:,0],
            embedding[:,1],
            c=af.labels_,
            s=0.1,
            cmap='Spectral')
plt.show()






100%|██████████| 4/4 [1:20:25<00:00, 1206.35s/it]


In [ ]:
#ARI, Mutual informaiton, Silhouette score

# open a file, where you stored the pickled data
file = open('C:\\Users\\Justin\\Documents\\GitHub\\PacManHuman\\Results\\affinityPropResultsMonkeyHWin10.pkl', 'rb')
# dump information to that file
data = pickle.load(file)

In [ ]:
#Get data after clustering and group for mean, variance, and joint kernel density maps
allvars=['speed','speedgrad','accelmag','prey1reldistance','prey1reldistancegrad','prey1relangularhead','prey1relangularheadgrad','prey1_scalar_projection','prey1_scalar_projection_gradient']
datvar=dict()
for varname in allvars:
    dattmp=[] #list of dataframes
    for i in range(n_sessions): #loop over sessions and retrieve data
        session=i+1
        dat=getvarforclustering(datall[str(session)]['p1trialdat'],varname,winlength)
        dattmp.append([pd.DataFrame(dat)])

    #Now combine all sessions
    datcombo=pd.DataFrame()

    for i in range(n_sessions):
        datcombo=pd.concat((datcombo,dattmp[i][0]),axis=1)

    datvar[varname]=datcombo



A=pd.DataFrame(datvar['prey1_scalar_projection'].to_numpy()[:,np.where(af.labels_==1)[0]].transpose().flatten())
B=pd.DataFrame(datvar['prey1_scalar_projection_gradient'].to_numpy()[:,np.where(af.labels_==1)[0]].transpose().flatten())

B.rename(columns={0:1},inplace=True)
sns.kdeplot(data=pd.concat((A,B),axis=1),x=0,y=1)
plt.show()

In [ ]:

#How to flatten out for plotting
datvar['prey1_scalar_projection'].to_numpy()[:,np.where(af.labels_==0)[0]].transpose().flatten()

In [ ]:
#Flat KDE plot
#plots of traces that are like distributional (you know, the ones like multiple reaches with shading as farther from mean at each point)
#Joint KDE plot